In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [28]:
pd.set_option('display.max_rows', None)

In [2]:
path_to_2d_objects_train = Path.cwd() / "resources" / "objects_2d_train.csv"
path_to_2d_objects_test = Path.cwd() / "resources" / "objects_2d_test.csv"
path_to_assigned_series = Path.cwd() / "resources" / "assigned_series_train.csv"

In [15]:
def get_train_df(path_to_train_objects, path_to_assigned):
    objects_train_df = pd.read_csv(path_to_train_objects)
    objects_train_df["series_uuid"] = objects_train_df.apply(lambda x: x["filename"].split(".nii.gz")[0], axis="columns")
    objects_train_df["lesion_label"] = objects_train_df.apply(lambda x: ",".join([x["lesion_type"], x["lesion_location"]]), axis="columns")
    assigned_df = pd.read_csv(path_to_assigned)
    objects_train_df = objects_train_df.merge(
        assigned_df,
        left_on="series_uuid",
        right_on="SeriesInstanceUID",
        how="left"
    )
    return objects_train_df

In [16]:
objects_train_df = get_train_df(path_to_2d_objects_train, path_to_assigned_series)
objects_test_df = pd.read_csv(path_to_2d_objects_test)

In [20]:
display(objects_train_df)

,filename,slice_idx,lesion_type,lesion_location,pixels_count,area_mm2,centroid_row,centroid_col,bbox_min_row,bbox_min_col,...,iteracion,id_paciente,code_paciente,id_estudio,StudyInstanceUID,id_serie,name_serie,SeriesInstanceUID,diagnóstico,Becado/a
0,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,22,m,pulmon,145,46.518326,64.089844,93.867188,108,156,...,1,12,12,20,1.3.51.0.1.1.172.19.3.128.2528604.2528543,62,Torax 1.5 I70f 2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,Cancer mama,Camilo Leal
1,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,23,m,pulmon,209,67.050552,64.358926,92.213106,106,152,...,1,12,12,20,1.3.51.0.1.1.172.19.3.128.2528604.2528543,62,Torax 1.5 I70f 2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,Cancer mama,Camilo Leal
2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,24,m,pulmon,251,80.524826,63.293078,91.356138,104,149,...,1,12,12,20,1.3.51.0.1.1.172.19.3.128.2528604.2528543,62,Torax 1.5 I70f 2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,Cancer mama,Camilo Leal
3,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,25,m,pulmon,315,101.057053,63.410528,91.480903,104,149,...,1,12,12,20,1.3.51.0.1.1.172.19.3.128.2528604.2528543,62,Torax 1.5 I70f 2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,Cancer mama,Camilo Leal
4,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,26,m,pulmon,323,103.623581,63.590061,91.231739,103,149,...,1,12,12,20,1.3.51.0.1.1.172.19.3.128.2528604.2528543,62,Torax 1.5 I70f 2,1.3.12.2.1107.5.1.4.83504.30000019041511214045...,Cancer mama,Camilo Leal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7660,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,141,m,suprarrenal,259,128.758007,140.936678,155.969270,190,213,...,1,37,37,139,1.3.51.0.1.1.172.19.3.128.3156833.3156772,430,Portal 5.0 Br38 1,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,Cancer pulmon,Camilo Leal
7661,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,142,m,suprarrenal,235,116.826763,140.829604,156.275316,190,214,...,1,37,37,139,1.3.51.0.1.1.172.19.3.128.3156833.3156772,430,Portal 5.0 Br38 1,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,Cancer pulmon,Camilo Leal
7662,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,146,n,abdomen,214,106.386925,164.975102,216.346963,225,300,...,1,37,37,139,1.3.51.0.1.1.172.19.3.128.3156833.3156772,430,Portal 5.0 Br38 1,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,Cancer pulmon,Camilo Leal
7663,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,147,n,abdomen,220,109.369736,165.023535,216.167338,224,299,...,1,37,37,139,1.3.51.0.1.1.172.19.3.128.3156833.3156772,430,Portal 5.0 Br38 1,1.3.12.2.1107.5.1.4.83885.30000023022112155417...,Cancer pulmon,Camilo Leal


In [24]:
# Evolution of total annotated 2d objects
grouped = objects_train_df.groupby(["iteracion"]).size().reset_index(name="annotated_2d_objects_train")
grouped["accumulated"] = grouped["annotated_2d_objects_train"].cumsum()
display(grouped)

,iteracion,annotated_2d_objects_train,accumulated
0,1,4216,4216
1,2,907,5123
2,3,2542,7665


In [31]:
# Evolution of annotated 2b objects grouped by lesion label
grouped = objects_train_df.groupby(["lesion_label", "iteracion"]).size().reset_index(name="annotated_2d_objects_train")
all_groups = grouped['lesion_label'].unique()
all_iterations = [1, 2, 3]  # All possible iteration values
multi_index = pd.MultiIndex.from_product([all_groups, all_iterations], names=['lesion_label', 'iteracion'])
# Reindex the grouped DataFrame to include all combinations
completed = grouped.set_index(['lesion_label', 'iteracion']).reindex(multi_index, fill_value=0).reset_index()
display(completed)

,lesion_label,iteracion,annotated_2d_objects_train
0,"m,higado",1,198
1,"m,higado",2,800
2,"m,higado",3,42
3,"m,pancreas",1,0
4,"m,pancreas",2,19
5,"m,pancreas",3,0
6,"m,pared abdominal",1,0
7,"m,pared abdominal",2,0
8,"m,pared abdominal",3,76
9,"m,pulmon",1,3858
